# 2장 k-최근접 이웃 알고리즘

- 학습 목표

k-최근접 이웃 알고리즘

텍스트 파일로부터 데이터 불러오기 및 구문 분석 

매스플롯라이브러리를 이용하여 scatter 플롯 생성 

수치형 값 정규화


## 2.1 거리 측정을 이용하여 분류하기

- **k-최근접 이웃 알고리즘**

장점: 높은 정확도, 오류 데이터(outlier)에 둔감, 데이터에 대한 가정이 없음  
단점: 계산 비용이 높음, 많은 메모리 요구  
적용: 수치형 값, 명목형 값  

1) 각 데이터와 그에 대한 레이블이 제공  
2) 레이블이 없는 새로운 데이터가 제공  
3) 새로운 데이터를 기존의 데이터와 가장 유사한 항목을 살펴봄(k개)  
4) k개의 데이터중 가장 유사한 데이터들 중 다수결을 통해 새로운 데이터의 레이블 결정  

---
예시

![image](figures/1.png)

![image](figures/2.png)

k=3일때, 모두 로맨스 이므로 새로운 영화의 레이블을 로멘스로 예측

---

- **kNN의 일반적인 접근 방법**  
1. 수집: 모든 방법  
2. 준비: 수치형 값(거리를 계산하기 위해 필요), 구조적으로 표현된 데이터 형식이 가장 좋다.  
3. 분석: 모든 방법  
4. 훈련: kNN 알고리즘은 적용되지 않음  
5. 검사: 오류율(error rate)을 계산함  
6. 사용: 응용 프로그램은 입력 데이터를 받아 구조가 있는 수치형 값으로 출력하는 데 필요하다. 그런 다음,
응용 프로그램은 입력 데이터를 가지고 kNN 알고리즘을 실행해서 입력 데이터가 속하는 분류 항목 을 결정한다. 응용 프로그램은 이때 계산된 분류 항목에서 몇 가지 조치를 취한다.  

### 2.1.1 준비: 파이썬으로 데이터 불러오기

In [1]:
from numpy import * 
import operator
def createDataSet():
    group = array([[1.0, 1.1], [1.0, 1.0], [0, 0], [0, 0.1]]) 
    labels = ['A', 'A', 'B', 'B']
    return group, labels

In [2]:
group, labels = createDataSet()
print(group)
print(labels)

[[ 1.   1.1]
 [ 1.   1. ]
 [ 0.   0. ]
 [ 0.   0.1]]
['A', 'A', 'B', 'B']


각 row는 데이터, 각 데이터에 대한 레이블

![image](figures/3.png)

### 2.1.2 kNN 분류 알고리즘 실행하기


In [3]:
def classify0(inX, dataSet, labels, k): 
    ##1. 거리 계산
    dataSetSize = dataSet.shape[0] ## 행 숫자 = 4
    diffMat = tile(inX,(dataSetSize,1)) - dataSet ## array([[0, 0],[0, 0],[0, 0],[0, 0]])
    sqDiffMat = diffMat ** 2 ##각 제곱
    sqDistances = sqDiffMat.sum(axis = 1) ## 가로로 더하기
    distances = sqDistances ** 0.5 ## 루트
    sortedDistIndicies = distances.argsort() 
    classCount={}
    ##2.가장 짧은 k거리를 투표
    for i in range(k):
        voteIlabel=labels[sortedDistIndicies[i]] 
        classCount[voteIlabel] = classCount.get(voteIlabel, 0) + 1
    ##3.아이템 정렬
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

- 유클리드 거리 사용

![image](figures/4.png)



In [4]:
classify0([0, 0], group, labels, 3)

'B'

### 2.1.3 분류기 검사하기

- 오류율 : 잘못분류/총분류

## 2.2 예제: kNN을 이용하여 데이트 사이트의 만남 주선 개선하기

- **예제: 데이트 사이트에서 얻은 결과에 kNN 사용하기**  
1. 수집: 제공된 텍스트 파일  
2. 준비: 파이썬에서 텍스트 파일 구문 분석하기  
3. 분석: 데이터를 2D 플롯(plots)으로 만들기 위해 매스플롯라이브러리 사용하기  
4. 훈련: kNN 알고리즘에는 적용되지 않음  
5. 검사: 헬렌이 준 검사용 예제 데이터의 일부를 사용하기 위한 함수를 작성한다. 검사용 예제는 검사에 사용되지 않는 예제에 반해 분류가 되어 있고, 예측된 분류 항목이 실질적인 분류 항목과 일치하지 않으면 오류로 계산한다.  
6. 사용: 헬렌이 입력한 몇 가지 데이터를 토대로 누구를 좋아하게 될 것인지를 예측하는 데 사용할 수 있는 간단한 커맨드 라인 프로그램을 구축한다.

### 2.2.1 준비: 텍스트 파일의 데이터 구문 분석하기

헬렌 데이터셋



In [5]:
def file2matrix(filename): 
    fr=open(filename)
    ##1. 파일 줄 수 구하기
    numberOfLines = len(fr.readlines())
    ##2. 반환하기 위한 Numpy행렬 생성하기
    returnMat = zeros((numberOfLines, 3)) 
    classLabelVector = []
    fr = open(filename, encoding="utf-8")
    index = 0
    ##3. 리스트의 줄 구문 분석하기
    for line in fr.readlines():
        line = line.strip()
        listFromLine = line.split('\t') #탭으로 구분
        returnMat[index, :] = listFromLine[0:3] 
        classLabelVector.append(int(listFromLine[-1]))
        index += 1
    return returnMat, classLabelVector

In [6]:
datingDataMat, datingLabels = file2matrix('datingTestSet2.txt')
datingDataMat

TypeError: 'encoding' is an invalid keyword argument for this function

In [ ]:
datingLabels[0:20]

### 2.2.2 분석: 매스플롯라이브러리로 scatter 플롯 생성하기

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from numpy import array
fig = plt.figure()
ax = fig.add_subplot(111)
## x=비디오 게임으로 보내는 시간 y=주당 아이스크림 소비량
ax.scatter(datingDataMat[:, 1], datingDataMat[:, 2], 15.0*array(datingLabels), 15.0*array(datingLabels))
plt.show()

![image](figures/5.png)

### 2.2.3 준비: 수치형 값 정규화하기

![image](figures/6.png)

newValue = (oldValue ‐ min) / (max ‐ min)


In [ ]:
def autoNorm(dataSet):
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    ranges = maxVals - minVals
    normDataSet = zeros(shape(dataSet)) 
    m = dataSet.shape[0]
    normDataSet = dataSet - tile(minVals, (1000, 1)) 
    normDataSet = normDataSet / tile(ranges, (1000, 1)) 
    return normDataSet, ranges, minVals

In [ ]:
normMat, ranges, minVals = autoNorm(datingDataMat)
normMat

In [ ]:
print(ranges)
print(minVals)

### 2.2.4 검사: 전체 프로그램으로 분류기 검사하기

- Train/Test set 나누기

- 오류율

In [ ]:
def datingClassTest(): 
    hoRatio = 0.10
    datingDataMat, datingLabels = file2matrix('datingTestSet2.txt') 
    normMat, ranges, minVals = autoNorm(datingDataMat)
    m = normMat.shape[0]
    numTestVecs = int(m * hoRatio)
    errorCount = 0.0
    for i in range(numTestVecs):
        classifierResult = classify0(normMat[i, :], normMat[numTestVecs:m, :], datingLabels[numTestVecs:m],3)
        print ("the classifier came back with: %d, the real answer is: %d" % (classifierResult, datingLabels[i]))
        if (classifierResult != datingLabels[i]): errorCount += 1.0 
    print ("the total error rate is: %f" % (errorCount/float(numTestVecs)))


In [ ]:
datingClassTest()

### 2.2.5 사용: 모두에게 유용한 시스템 만들기



In [ ]:
def classifyPerson():
    resultList = ['not at all', 'in small doses', 'in large doses']
    percentTats = float(input(\
                                  "percentage of time spent playing video games?"))
    ffMiles = float(input("frequent flier miles earned per year?"))
    iceCream = float(input("liters of ice cream consumed per year?"))
    datingDataMat, datingLabels = file2matrix('datingTestSet2.txt')
    normMat, ranges, minVals = autoNorm(datingDataMat)
    inArr = array([ffMiles, percentTats, iceCream, ])
    classifierResult = classify0((inArr - \
                                  minVals)/ranges, normMat, datingLabels, 3)
    print ("You will probably like this person: %s" % resultList[classifierResult - 1])
    


In [ ]:
classifyPerson()

## 2.3 예제: 필기체 인식 시스템

![image](figures/7.png)

- **예제: 필기체 인식 시스템에 kNN 사용하기**
1. 수집: 제공된 텍스트 파일  
2. 준비: 분류기 classify0()을 사용해서 이미지 형태를 리스트 형태로 전환하는 함수 작성  
3. 분석: 올바른 데이터인지 확인하기 위해 파이썬 쉘에서 준비된 데이터 관찰  
4. 훈련: kNN 알고리즘에는 적용되지 않음  
5. 검사: 검사용 예제에 있는 데이터의 일부분을 사용하여 함수 만들기. 검사에 사용되지 않을 예제로부터 검사에 사용될 예제를 분류해 낸다. 예측된 분류 항목이 실제 분류 항목과 일치하지 않는다면, 오류로 간주하고 오류 횟수를 누적한다.  
6. 사용: 이 예제에서는 다루지 않는다. 미국에서 메일을 정렬하기 위해 사용되는 시스템처럼 이미지로부터번호를 추출하기 위한 프로그램을 만들 수 있다.  

### 2.3.1 준비: 이미지를 검사 벡터로 변환하기

In [ ]:
def img2vector(filename): 
    returnVect = zeros((1, 1024)) 
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline() 
        for j in range(32):
            returnVect[0, 32*i+j] = int(lineStr[j]) 
    return returnVect


In [ ]:
testVector = img2vector('testDigits/0_13.txt')

In [ ]:
testVector[0, 0:31]

In [ ]:
testVector[0, 32:63]

### 2.3.2 검사: 필기체 번호에 kNN 적용하기



In [ ]:
from os import listdir
def handwritingClassTest(): 
    hwLabels=[]
    ## 1. 디렉토리 명을 구함
    trainingFileList = listdir('trainingDigits') 
    m = len(trainingFileList)
    trainingMat = zeros((m, 1024))
    ## 2. 파일명에서 분류 번호를 처리함
    for i in range(m):
        fileNameStr = trainingFileList[i] 
        fileStr = fileNameStr.split('.')[0] 
        classNumStr = int(fileStr.split('_')[0]) 
        hwLabels.append(classNumStr)
        trainingMat[i,:] = img2vector('trainingDigits/%s' % fileNameStr)
    testFileList = listdir('testDigits')        #iterate through the test set
    errorCount = 0.0
    mTest = len(testFileList)
    for i in range(mTest):
        fileNameStr = testFileList[i]
        fileStr = fileNameStr.split('.')[0]     #take off .txt
        classNumStr = int(fileStr.split('_')[0])
        vectorUnderTest = img2vector('testDigits/%s' % fileNameStr)
        classifierResult = classify0(vectorUnderTest, trainingMat, hwLabels, 3)
        print ("the classifier came back with: %d, the real answer is: %d" % (classifierResult, classNumStr))
        if (classifierResult != classNumStr): errorCount += 1.0
    print ("\nthe total number of errors is: %d" % errorCount)
    print ("\nthe total error rate is: %f" % (errorCount/float(mTest)))
        

In [ ]:
handwritingClassTest()

이 알고리즘은 속도가 느린 편 - 2000번의 거리계산이 일어남 - KNN의 변형인 kD-트리를 통하여 계산횟수를 줄일 수 있다.

## 2.4 요약

- k최근접 이웃 알고리즘

사례 기반 학습

데이터 집합 전체 - 큰 저장소 필요

모든 데이터에 대하여 거리 계산 - 크기가 커지면 오래걸림

데이터 구조에 대한 정보 없음(평균/모범적 사례 식별 불가)